In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sqlite3
import numpy as np
from SlideRunner.dataAccess.database import Database
from tqdm import tqdm
from pathlib import Path
import openslide
import time
import pickle
import cv2
from Detection.data_loader import *

In [3]:
path = Path('/data/Datasets/EIPH_WSI/')

database = Database()
database.open(str(path/'EIPH.sqlite'))

size = 1024
level = 0

In [4]:
files = []
lbl_bbox = []

getslides = """SELECT uid, filename FROM Slides"""
for currslide, filename in tqdm(database.execute(getslides).fetchall()):
    database.loadIntoMemory(currslide)

    check = True if 'erliner' in filename else False
    slidetype = 'Berliner Blau/' if check else 'Turnbull Blue/'

    slide_path = path / slidetype / filename

    slide = openslide.open_slide(str(slide_path))
    level = level#slide.level_count - 1
    level_dimension = slide.level_dimensions[level]
    down_factor = slide.level_downsamples[level]

    classes = {3: 0, 4: 1, 5: 2, 6: 3, 7: 4}#{3: 1, 4: 2, 5: 3, 6: 4, 7: 5} #
    labels, bboxes = [], []
    for id, annotation in database.annotations.items():
        if annotation.labels[0].classId in classes:
            d = 2 * annotation.r / down_factor
            x_min = (annotation.x1 - annotation.r) / down_factor
            y_min = (annotation.y1 - annotation.r) / down_factor
            x_max = x_min + d
            y_max = y_min + d
            label = classes[annotation.labels[0].classId]

            bboxes.append([int(x_min), int(y_min), int(x_max), int(y_max)])
            labels.append(label)

    if len(bboxes) > 0:
        lbl_bbox.append([bboxes, labels])
        files.append(SlideContainer(file=slide_path, y=[bboxes, labels],
                                    level=level, width=size, height=size))

  0%|          | 0/24 [00:00<?, ?it/s]

Loading DB into memory ...


  4%|▍         | 1/24 [00:00<00:07,  3.13it/s]

Loading DB into memory ...


  8%|▊         | 2/24 [00:01<00:14,  1.52it/s]

...

Loading DB into memory ...
Loading DB into memory ...
Loading DB into memory ...


 88%|████████▊ | 21/24 [00:06<00:00,  4.26it/s]

Loading DB into memory ...
Loading DB into memory ...


100%|██████████| 24/24 [00:07<00:00,  3.35it/s]

Loading DB into memory ...


### Fläche 

In [6]:
for file in files:
    print("File: {0} X: {1} Y: {2}".format(file.file, file.slide.level_dimensions[0][0], file.slide.level_dimensions[0][1]))

File: /data/Datasets/EIPH_WSI/Berliner Blau/01_EIPH_563479 Berliner Blau.svs X: 35999 Y: 37162
File: /data/Datasets/EIPH_WSI/Berliner Blau/02_EIPH_574162 berliner blau.svs X: 30599 Y: 31244
File: /data/Datasets/EIPH_WSI/Berliner Blau/03_EIPH_566933 R Berliner Blau neu.svs X: 30599 Y: 30603
File: /data/Datasets/EIPH_WSI/Berliner Blau/04_EIPH_567017 Berliner Blau.svs X: 34200 Y: 32037
File: /data/Datasets/EIPH_WSI/Berliner Blau/05_EIPH_569923 Berliner Blau.svs X: 41399 Y: 38591
File: /data/Datasets/EIPH_WSI/Berliner Blau/27_EIPH_571557 berliner blau.svs X: 32399 Y: 30918
File: /data/Datasets/EIPH_WSI/Berliner Blau/31_EIPH_568354 Berliner Blau.svs X: 34200 Y: 34761
File: /data/Datasets/EIPH_WSI/Berliner Blau/11_EIPH_575697 Berliner Blau.svs X: 35999 Y: 34276
File: /data/Datasets/EIPH_WSI/Turnbull Blue/29_EIPH_566481 L Turnbull blue.svs X: 35999 Y: 34118
File: /data/Datasets/EIPH_WSI/Turnbull Blue/28_EIPH_569948 L Turnbull blue.svs X: 35999 Y: 32515
File: /data/Datasets/EIPH_WSI/Turnbull B

In [100]:
total_pixel = {}
µm_pixel = 0.25
for file in files:
    x = file.slide.level_dimensions[0][0]
    y = file.slide.level_dimensions[0][1]
    total_pixel[file.file.name] = (x * y) / 1000000 * (µm_pixel * µm_pixel) # / µm zu mm

In [102]:
total_pixel

{'01_EIPH_563479 Berliner Blau.svs': 83.612177375,
 '02_EIPH_574162 berliner blau.svs': 59.75219725,
 '03_EIPH_566933 R Berliner Blau neu.svs': 58.5263248125,
 '04_EIPH_567017 Berliner Blau.svs': 68.4790875,
 '05_EIPH_569923 Berliner Blau.svs': 99.8518005625,
 '27_EIPH_571557 berliner blau.svs': 62.607017625,
 '31_EIPH_568354 Berliner Blau.svs': 74.3016375,
 '11_EIPH_575697 Berliner Blau.svs': 77.11885775,
 '29_EIPH_566481 L Turnbull blue.svs': 76.763367625,
 '28_EIPH_569948 L Turnbull blue.svs': 73.1567178125,
 '14_EIPH_568381 Turnbull blue.svs': 78.5678175,
 '20_EIPH_576150 berliner blau.svs': 84.7571701875,
 '07_EIPH_574882 left lung Turnbull blue.svs': 74.95891775,
 '08_EIPH_574999 R Turnbull blue.svs': 77.47884775,
 '09_EIPH_563417 L Turnbull blue.svs': 75.3166578125,
 '22_EIPH_575216 Turnbull blue.svs': 68.7770021875,
 '26_EIPH_566482 L Berliner Blau.svs': 72.2496375}

In [94]:
total_pixel = np.array(list(total_pixel.values()))

In [95]:
total_pixel.sum()

1266.2752365

In [96]:
total_pixel.mean()

74.48677861764705

In [97]:
total_pixel.std()

9.577063858340159

### Label

In [52]:
img2bbox = dict(zip(files, np.array(lbl_bbox)))
get_y_func = lambda o:img2bbox[o]

In [62]:
data = {'File':[],
        'Total':[],
        'Mean':[],
        'Std':[],
        0:[],
        1:[],
        2:[],
        3:[],
        4:[],
       }

for file in files:
    data['File'].append(file.file.name)
    
    boxes, labels = get_y_func(file)
    labels = np.array(labels)
    data['Total'].append(len(labels))
    data['Mean'].append(labels.mean())
    data['Std'].append(labels.std())
    
    for i in range(5):
        data[i].append(len(labels[labels == i]))
    
df = pd.DataFrame(data)

In [63]:
df

,File,Total,Mean,Std,0,1,2,3,4
0,01_EIPH_563479 Berliner Blau.svs,4446,1.260009,0.958827,1013,1782,1218,348,85
1,02_EIPH_574162 berliner blau.svs,12812,0.728926,0.681335,5084,6203,1450,64,11
2,03_EIPH_566933 R Berliner Blau neu.svs,6325,0.374071,0.584239,4295,1697,330,3,0
3,04_EIPH_567017 Berliner Blau.svs,5448,0.628671,0.656466,2551,2379,508,10,0
4,05_EIPH_569923 Berliner Blau.svs,2489,0.336681,0.554706,1754,634,99,2,0
5,27_EIPH_571557 berliner blau.svs,5751,0.590680,0.647408,2839,2452,435,25,0
6,31_EIPH_568354 Berliner Blau.svs,3143,0.394209,0.547628,1995,1062,81,5,0
7,11_EIPH_575697 Berliner Blau.svs,1841,1.479631,0.865001,283,553,859,131,15
8,29_EIPH_566481 L Turnbull blue.svs,968,0.433884,0.672488,637,252,70,8,1
9,28_EIPH_569948 L Turnbull blue.svs,1112,0.348921,0.551827,767,302,43,0,0


In [76]:
df['Total'].mean()

4591.0

In [77]:
df['Total'].std()

3389.839929849196

In [72]:
df.sum(axis = 0, skipna = True, numeric_only=True) 

Total    78047.000000
Mean        16.449628
Std         13.103458
0        29055.000000
1        26925.000000
2        13524.000000
3         7070.000000
4         1473.000000
dtype: float64

In [71]:
df[0]

0     1013
1     5084
2     4295
3     2551
4     1754
5     2839
6     1995
7      283
8      637
9      767
10     568
11    2611
12    1908
13      48
14     471
15    1639
16     592
Name: 0, dtype: int64

In [73]:
inference_data = pickle.load(open("inference_results_boxes.p", "rb"))

In [75]:
inference_data

{'01_EIPH_563479 Berliner Blau.svs': [array([6.480000e+02, 1.683700e+04, 7.210000e+02, 1.689500e+04, 2.000000e+00, 2.606018e-01, 1.606188e+00]),
  array([6.090000e+02, 1.685400e+04, 7.510000e+02, 1.696800e+04, 2.000000e+00, 7.846485e-01, 1.364782e+00]),
  array([6.380000e+02, 1.689600e+04, 7.190000e+02, 1.697100e+04, 2.000000e+00, 7.165377e-01, 1.519179e+00]),
  array([5.830000e+02, 1.689600e+04, 7.600000e+02, 1.702000e+04, 2.000000e+00, 1.477406e-01, 1.419886e+00]),
  array([3.400000e+02, 1.810800e+04, 4.500000e+02, 1.823700e+04, 0.000000e+00, 1.248962e-01, 1.743477e+00]),
  array([2.830000e+02, 1.807600e+04, 5.010000e+02, 1.829700e+04, 0.000000e+00, 1.002470e-01, 1.672305e+00]),
  array([3.400000e+02, 1.810800e+04, 4.500000e+02, 1.823800e+04, 0.000000e+00, 1.248611e-01, 1.622688e+00]),
  array([2.83000e+02, 1.80760e+04, 5.01000e+02, 1.82970e+04, 0.00000e+00, 1.00514e-01, 1.54851e+00]),
  array([7.550000e+02, 1.922900e+04, 8.640000e+02, 1.935900e+04, 3.000000e+00, 7.445822e-01, 1.5145